In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!pip install ipython-autotime
%load_ext autotime

  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
time: 3.49 ms (started: 2024-08-19 15:42:41 +00:00)


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
time: 34.9 s (started: 2024-08-19 15:42:41 +00:00)


### 1. Combine the data from p14 and p9

In [4]:
import shutil
import pandas as pd

# Reading the CSV files
data_p9 = pd.read_csv('/content/drive/MyDrive/THESIS/thesis_data/THESIS_DATA/spectrogram_images_dataset_p9/spectrogram_images_dataset_p9.csv')
data_p14 = pd.read_csv('/content/drive/MyDrive/THESIS/thesis_data/THESIS_DATA/spectrogram_images_dataset_p14/spectrogram_images_dataset_p14.csv')

# Concatenating the two DataFrames
data_combined = pd.concat([data_p9, data_p14])

# Saving the combined data to a new CSV file in the Drive
output_path = '/content/drive/MyDrive/THESIS/thesis_data/THESIS_DATA/spectrogram_images_dataset_p14/spectrogram_images_dataset_combined.csv'
data_combined.to_csv(output_path, index=False)

time: 10.7 s (started: 2024-08-19 15:43:16 +00:00)


## Build and Train Model

In [5]:
csv_file_path = "/content/drive/MyDrive/THESIS/thesis_data/THESIS_DATA/spectrogram_images_dataset_p14/spectrogram_images_dataset_combined.csv"

time: 1.51 ms (started: 2024-08-19 15:43:27 +00:00)


In [7]:
import os
import csv
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.optimizers import Adam

# Parameters
image_size = (128, 128)  # Resize all images to this size
test_size = 0.2

# Load CSV file
data = pd.read_csv(csv_file_path)

# Extract image paths and labels
image_paths = data['image_path'].values
labels = data['label'].values

# Encode labels
le = LabelEncoder()
encoded_labels = le.fit_transform(labels)
categorical_labels = to_categorical(encoded_labels)

# Load and preprocess images
images = []
for img_path in image_paths:
    img = load_img(img_path, target_size=image_size, color_mode='rgb')
    img_array = img_to_array(img) / 255.0  # Normalize to [0, 1]
    images.append(img_array)

images = np.array(images)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, categorical_labels, test_size=test_size, random_state=42)

time: 13.3 s (started: 2024-08-19 16:01:46 +00:00)


###Trying Keras CNN

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the model
model = Sequential()

# Convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the output and add dense layers
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(le.classes_), activation='softmax'))  # Output layer with softmax

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │       3,211,392 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 7)                   │             903 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,305,543 (12.61 MB)

 Trainable params: 3,305,543 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

time: 162 ms (started: 2024-08-19 16:15:26 +00:00)


In [9]:
from tensorflow.keras.callbacks import EarlyStopping

# Define early stopping
early_stopping = EarlyStopping(monitor='val_accuracy',  # Monitor validation loss
                               patience=10,          # Number of epochs with no improvement
                               restore_best_weights=True)

time: 2.99 ms (started: 2024-08-19 16:05:45 +00:00)


In [10]:
# Train the model with early stopping
history = model.fit(X_train, y_train,
                    epochs=200,
                    batch_size=32,
                    validation_split=0.2,
                    callbacks=[early_stopping])

Epoch 1/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 30s 1s/step - accuracy: 0.1282 - loss: 1.9520 - val_accuracy: 0.1158 - val_loss: 1.9510
Epoch 2/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 23s 977ms/step - accuracy: 0.1585 - loss: 1.9452 - val_accuracy: 0.1158 - val_loss: 1.9504
Epoch 3/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 26s 1s/step - accuracy: 0.1262 - loss: 1.9476 - val_accuracy: 0.1158 - val_loss: 1.9491
Epoch 4/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 24s 1s/step - accuracy: 0.1567 - loss: 1.9437 - val_accuracy: 0.1158 - val_loss: 1.9505
Epoch 5/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.1341 - loss: 1.9465 - val_accuracy: 0.1158 - val_loss: 1.9486
Epoch 6/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 41s 998ms/step - accuracy: 0.1259 - loss: 1.9470 - val_accuracy: 0.1158 - val_loss: 1.9482
Epoch 7/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 45s 1s/step - accuracy: 0.1148 - loss: 1.9483 - val_accuracy: 0.1158 - val_loss: 1.9478
Epoch 8/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 37s 976ms/step - accuracy: 0.1563 - loss: 1.9443 - val_accuracy: 0

In [11]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy:.4f}")
print(f"Test loss: {test_loss:.4f}")

# Save the model
# model.save('spectrogram_classification_model.h5')

8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 229ms/step - accuracy: 0.1147 - loss: 1.9452
Test accuracy: 0.1392
Test loss: 1.9443
time: 1.94 s (started: 2024-08-19 16:12:01 +00:00)


Without early stopping callback

In [13]:
# Train the model with early stopping
history = model.fit(X_train, y_train,
                    epochs=200,
                    batch_size=32,
                    validation_split=0.2)

Epoch 1/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - accuracy: 0.1539 - loss: 1.9513 - val_accuracy: 0.1474 - val_loss: 1.9466
Epoch 2/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 43s 1s/step - accuracy: 0.1427 - loss: 1.9490 - val_accuracy: 0.1526 - val_loss: 1.9493
Epoch 3/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 36s 956ms/step - accuracy: 0.1702 - loss: 1.9422 - val_accuracy: 0.1526 - val_loss: 1.9485
Epoch 4/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 42s 988ms/step - accuracy: 0.1722 - loss: 1.9442 - val_accuracy: 0.1474 - val_loss: 1.9471
Epoch 5/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 41s 972ms/step - accuracy: 0.1484 - loss: 1.9455 - val_accuracy: 0.1158 - val_loss: 1.9479
Epoch 6/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 41s 964ms/step - accuracy: 0.1565 - loss: 1.9440 - val_accuracy: 0.1158 - val_loss: 1.9511
Epoch 7/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 45s 1s/step - accuracy: 0.1462 - loss: 1.9468 - val_accuracy: 0.1158 - val_loss: 1.9554
Epoch 8/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 37s 962ms/step - accuracy: 0.1327 - loss: 1.9497 - val_accur

In [14]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy:.4f}")
print(f"Test loss: {test_loss:.4f}")

8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 233ms/step - accuracy: 0.2554 - loss: 3.8190
Test accuracy: 0.2489
Test loss: 4.0766
time: 2.19 s (started: 2024-08-19 18:17:43 +00:00)
